# NLP_GoingDeeper | P05.Transformer_Translation_Kor2Eng
---
- 한국어 문장을 입력으로 하여 영어로 번역된 문장을 출력하는 번역기를 만들어봅니다. 

- [jungyeul/korean-parallel-corpora](https://github.com/jungyeul/korean-parallel-corpora/tree/master/korean-english-news-v1)에서 제공하는 데이터셋을 활용합니다.

In [1]:
# ! ln -s ~/data ~/aiffel/GoingDeeper/DATA/transformer/data

In [2]:
#- 나눔글꼴 설치
! sudo apt -qq -y install fonts-nanum

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [3]:
#- SentencePiece 설치

! pip install sentencepiece

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [4]:
import matplotlib as mpl 
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

import matplotlib.font_manager as fm
fontpath = '/user/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
mpl.font_manager._rebuild()

In [5]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split

import sentencepiece as spm

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

print(tf.__version__)

2.4.1


In [6]:
data_dir = os.getenv('HOME')+'/aiffel/GoingDeeper/DATA/transformer/data'
kor_path = data_dir+"/korean-english-park.train.ko"
eng_path = data_dir+"/korean-english-park.train.en"

In [7]:
with open(kor_path, "r") as f: kor = f.read().splitlines()
with open(eng_path, "r") as f: eng = f.read().splitlines()
print("Data Size:({},{})".format(len(kor), len(eng)))
print("Example:")

cnt = 0
for ko, en in zip(kor, eng):
    print(f">>\n한국어 : {ko}\n영어 : {en}\n")
    cnt += 1
    if cnt == 5:
        break

Data Size:(94123,94123)
Example:
>>
한국어 : 개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"
영어 : Much of personal computing is about "can you top this?"

>>
한국어 : 모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.
영어 : so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.

>>
한국어 : 그러나 이것은 또한 책상도 필요로 하지 않는다.
영어 : Like all optical mice, But it also doesn't need a desk.

>>
한국어 : 79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.
영어 : uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.

>>
한국어 : 정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.
영어 : Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror atta

# 2. 데이터 정제 및 토큰화 
### 1) 중복데이터 제거
- set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거합니다. 
- 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.

In [8]:
# 데이터 정제 및 토큰화
def clean_corpus(kor, eng):
    assert len(kor) == len(eng)
    cleaned_corpus = list(set(zip(kor, eng)))

    return cleaned_corpus

In [9]:
cleaned_corpus = clean_corpus(kor, eng)
len(cleaned_corpus)

78968

### 2) 정제함수 정의

- 모든 입력을 소문자로 변환합니다.
- 알파벳, 문장부호, 한글만 남기고 모두 제거합니다.
- 문장부호 양옆에 공백을 추가합니다.
- 문장 앞뒤의 불필요한 공백을 제거합니다.

In [10]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() #- 소문자 변환
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣?.!,]+", " ", sentence) #- 알파벳, 문장부호, 한글 이외는 제거
    sentence = re.sub(r'[" "]+', " ", sentence) #- 여러개의 공백은 하나의 공백으로 바꾸기
    sentence = sentence.strip() #- 양쪽 공백 제거
    return sentence

### 3) 토큰화
- 한글 말뭉치 kor_corpus 와 영문 말뭉치 eng_corpus 를 각각 분리한 후, 정제하여 토큰화를 진행합니다. 
- 토큰화에는 [SentencePiece](https://github.com/google/sentencepiece)를 사용합니다. 
    - generate_tokenizer() 함수를 정의하여 최종적으로 ko_tokenizer와 en_tokenizer를 얻습니다. 
    - en_tokenizer에는 set_encode_extra_options("bos:eos") 함수를 실행해 타겟 입력이 문장의 시작 토큰과 끝 토큰을 포함할 수 있게 합니다.
    - 단어 사전을 매개변수로 받아 원하는 크기의 사전을 정의할 수 있게 합니다. (기본: 20,000)
    - 학습 후 저장된 model 파일을 SentencePieceProcessor() 클래스에 Load()한 후 반환합니다.
    - 특수 토큰의 인덱스를 아래와 동일하게 지정합니다.
        - \<PAD> : 0 / \<BOS\> : 1 / \<EOS\> : 2 / \<UNK\> : 3
- [spm의 입력은 txt로 넣어야 합니다.](https://lovit.github.io/nlp/2018/04/02/wpm/)

In [11]:
# Sentencepiece를 활용하여 학습한 tokenizer를 생성합니다.
def generate_tokenizer(corpus, model_type='unigram', vocab_size=32000, lang="ko"): 
    
    input_file = f'{lang}_spm_input.txt'

    with open(input_file, 'w', encoding='utf-8') as f:
        for sent in corpus:
            f.write('{}\n'.format(sent))
        
    sp_model_root='sentencepiece'
    if not os.path.isdir(sp_model_root): os.mkdir(sp_model_root)
    
    vocab_size = vocab_size # vocab 사이즈
    prefix = 'tokenizer_%s_%s' % (lang,model_type+str(vocab_size))
    prefix = os.path.join(sp_model_root, prefix) # 저장될 tokenizer 모델에 붙는 이름
    pad_id=0
    bos_id=1 #<start> token을 1으로 설정
    eos_id=2 #<end> token을 2으로 설정
    unk_id=3 #<unknown> token을 3으로 설정
    character_coverage = 1.0 # to reduce character set 
    # model_type = model_type # Choose from unigram (default), bpe, char, or word
    input_argument = '--input=%s --pad_id=%s --bos_id=%s --eos_id=%s --unk_id=%s --model_prefix=%s --vocab_size=%s --character_coverage=%s --model_type=%s'
    cmd = input_argument%(input_file, pad_id, bos_id, eos_id, unk_id, prefix, vocab_size, character_coverage, model_type)

#     cmd = templates.format(corpus,
#                 pad_id,
#                 bos_id,
#                 eos_id,
#                 unk_id,
#                 prefix,
#                 vocab_size,
#                 character_coverage,
#                 model_type)
                       
    spm.SentencePieceTrainer.Train(cmd)
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f'{prefix}.model')
    
    return tokenizer

In [12]:
cleaned_corpus[1]

('그는 당시 자녀양육비를 내지 못해 경찰에 수배 중이었으며 경찰은 그의 소재를 파악하지 못하고 있었다.',
 'He was being sought on a parole violation for failure to pay child support, but police who have said they want to question him further had been unable to locate him.')

In [13]:
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 32000

eng_corpus = []
kor_corpus = []

for pair in cleaned_corpus:
    k, e = pair[0],pair[1]

    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

ko_tokenizer = generate_tokenizer(kor_corpus, 'unigram', SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, 'unigram', TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

True

In [14]:
ko_test_txts = ['세계기상기구는 “코로나19에 따른 경제 활동 둔화로 탄소 배출이 일시적으로 줄었지만, 대기 중 이산화탄소 농도에 뚜렷한 영향을 주지 못했다”고 설명했다.','기후변화 대응·적응에 필요한 재원 분담 과정에서 나타나는 선진국과 개발도상국의 이견도 오래 묵은 논쟁거리다.']
en_test_txts = ['Democrats, including President Biden, are lobbying for Senator Joe Manchin’s support, knowing he is a crucial swing vote on their domestic agenda.','The contest to fill a vacant State House seat in South Texas has exposed the vulnerabilities of a Democratic stronghold.']


In [15]:
def testSP(sp, txts):
    tokens_list = []
    for txt in txts:
        tokens = sp.encode_as_pieces(txt)
        #ids = sp.encode_as_ids(txt)
        tokens_list.append(tokens)
    return tokens_list

In [16]:
testSP(ko_tokenizer, ko_test_txts)

[['▁세계',
  '기',
  '상',
  '기구',
  '는',
  '▁',
  '“',
  '코',
  '로',
  '나',
  '19',
  '에',
  '▁따른',
  '▁경제',
  '▁활동',
  '▁둔화',
  '로',
  '▁탄소',
  '▁배출',
  '이',
  '▁일시적',
  '으로',
  '▁줄',
  '었지만',
  ',',
  '▁대기',
  '▁중',
  '▁이산화탄소',
  '▁농도',
  '에',
  '▁뚜렷',
  '한',
  '▁영향',
  '을',
  '▁주지',
  '▁못했다',
  '”',
  '고',
  '▁설명했다',
  '.'],
 ['▁기후변화',
  '▁대응',
  '·',
  '적',
  '응',
  '에',
  '▁필요한',
  '▁재원',
  '▁분담',
  '▁과정에서',
  '▁나타나',
  '는',
  '▁선진국',
  '과',
  '▁개발도상국',
  '의',
  '▁이견',
  '도',
  '▁오래',
  '▁묵',
  '은',
  '▁논쟁거리',
  '다',
  '.']]

In [17]:
testSP(en_tokenizer, en_test_txts)

[['<s>',
  '▁',
  'D',
  'em',
  'ocrats',
  ',',
  '▁including',
  '▁',
  'P',
  'res',
  'ident',
  '▁',
  'B',
  'iden',
  ',',
  '▁are',
  '▁lobby',
  'ing',
  '▁for',
  '▁',
  'S',
  'en',
  'ator',
  '▁',
  'J',
  'o',
  'e',
  '▁',
  'M',
  'an',
  'chi',
  'n',
  '’',
  's',
  '▁support',
  ',',
  '▁knowing',
  '▁he',
  '▁is',
  '▁a',
  '▁crucial',
  '▁swing',
  '▁vote',
  '▁on',
  '▁their',
  '▁domestic',
  '▁agenda',
  '.',
  '</s>'],
 ['<s>',
  '▁',
  'T',
  'he',
  '▁contest',
  '▁to',
  '▁fill',
  '▁a',
  '▁vacant',
  '▁',
  'S',
  'tate',
  '▁',
  'H',
  'ouse',
  '▁seat',
  '▁in',
  '▁',
  'S',
  'out',
  'h',
  '▁',
  'T',
  'ex',
  'as',
  '▁has',
  '▁exposed',
  '▁the',
  '▁vulnerabilities',
  '▁of',
  '▁a',
  '▁',
  'D',
  'e',
  'mo',
  'cratic',
  '▁stronghold',
  '.',
  '</s>']]

### 4) 데이터 선별
- 토크나이저를 활용해 토큰의 길이가 50 이하인 데이터를 선별하여 src_corpus 와 tgt_corpus 를 각각 구축합니다. 
- 이후 텐서 enc_train 과 dec_train 으로 변환합니다. 

In [ ]:
from tqdm.notebook import tqdm    # Process 과정을 보기 위해

src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰의 길이가 50 이하인 문장만 남깁니다. 
for idx in tqdm(range(len(kor_corpus))):
    m

# 패딩처리를 완료하여 학습용 데이터를 완성합니다. 
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

# 3. 모델 설계


# 4. 훈련
### 1) 모델 선언
2 Layer를 가지는 Transformer를 선언합니다. (하이퍼파라미터는 자유롭게 조절) 

In [ ]:
transformer = # [[YOUR CODE]]

### 2) Learning Rate Scheduler
- 논문에서 사용한 것과 동일한 Learning Rate Scheduler를 선언하고, 이를 포함하는 Adam Optimizer를 선언합니다. 

In [ ]:
learning_rate = # [[YOUR CODE]]
optimizer = # [[YOUR CODE]]

### 3) Loss 함수 정의
- Sequence-to-sequence 모델에서 사용했던 Loss와 유사하되, Masking 되지 않은 입력의 개수로 Scaling하는 과정을 추가합니다. (트랜스포머가 모든 입력에 대한 Loss를 한 번에 구하기 때문입니다.)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

### 4) train_step 함수 정의
- 입력 데이터에 알맞은 Mask를 생성하고, 이를 모델에 전달하여 연산에서 사용할 수 있게 합니다. 

In [ ]:
# Train Step 함수 정의

@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]
        
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행합니다.
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions[:, :-1])

    # 최종적으로 optimizer.apply_gradients()가 사용됩니다. 
    # [[YOUR CODE]]
    
    return loss, enc_attns, dec_attns, dec_enc_attns

### 5) 학습 진행
- 매 Epoch 마다 제시된 예문에 대한 번역을 생성하고, 멋진 번역이 생성되면 그때의 하이퍼파라미터와 생성된 번역을 기록합니다. 

- 예문
    1. 오바마는 대통령이다.
    2. 시민들은 도시 속에 산다.
    3. 커피는 필요 없다.
    4. 일곱 명의 사망자가 발생했다.
    

In [ ]:
# 학습

from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 20

examples = [
            "오바마는 대통령이다.",
            "시민들은 도시 속에 산다.",
            "커피는 필요 없다.",
            "일곱 명의 사망자가 발생했다."
]

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    for example in examples:
        translate(example, transformer, ko_tokenizer, en_tokenizer)
Copyright

In [ ]:
# Attention 시각화 함수

def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn
        seaborn.heatmap(data, 
                        square=True,
                        vmin=0.0, vmax=1.0, 
                        cbar=False, ax=ax,
                        xticklabels=x,
                        yticklabels=y)
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], src, src)
        plt.show()
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer+1)
        for h in range(4):
            draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer+1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], src, tgt)
        plt.show()

In [ ]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [ ]:
# 번역 생성 및 Attention 시각화 결합

def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

# 4. 결과
### Translations
1. 오바마는 대통령이다. > 
2. 시민들은 도시 속에 산다. > 
3. 커피는 필요 없다. > 
4. 일곱 명의 사망자가 발생했다. > 

### Hyperparameters
- n_layers: 
- d_model: 
- n_heads: 
- d_ff: 
- dropout: 

### Training Parameters
- Warmup Steps: 
- Batch Size: 
- Epoch At: 